In [1]:
#load "sig.fsx"
open CSCI374.ExtraReflection

The below script needs to be able to find the current output cell; this is an easy method to get it.

# F# Programming - Pattern Matching

## Pattern matching for flow of control

Most imperative languages offer a variety of control flow statements for branching and looping:

- if-then-else (and the ternary version bool ? if-true : if-false)
- case or switch statements
- for and foreach loops, with break and continue
- and even the dreaded goto

F# does support some of these, but F# also supports the most general form of conditional expression, which is **pattern-matching**. The pattern matching construct in F# allows you to **pattern match over a variety of types and values**.

In [2]:
let booleanExpression = true

(if booleanExpression then 10 else 20) + 1

11

In [3]:
if booleanExpression
    then "TRUE"
    else "FALSE"

TRUE

A typical matching expression that replaces `if-then-else` looks like this:

In [4]:
match booleanExpression with
| true -> "TRUE" // true branch
| false -> "FALSE" // false branch

TRUE

In [5]:
// This function calculates the nth term in the Fibonacci sequence.
let rec fib n = 
    if n = 1 then 1
    else if n = 2 then 1
    else fib(n-1) + fib(n-2)
        
sgn fib |> printfn "%s"
        
fib 11

Function: n:int -> int


89

In [28]:
let rec fib n =
    match n with
    | 1 -> 1
    | 2 -> 1
    | nth -> fib(nth - 1) + fib(nth - 2)

sgn fib

Function: n:int -> int

In [ ]:
fib 12

### Patterns

In general the syntax is:

```fsharp
match e_0 with 
| p_1 when g_1 -> e_1
| p_2 -> e_2
....
| p_n -> e_n
| _   -> e_n+1
```        

Semantics:        
        
- The keyword `match` followed by the **identifier** that will be matched and then the keyword `with`.
- All the possible **matching rules** separated by vertical bars, `|`.
- A **rule** consists of either a *constant* or an *identifier*, followed by an arrow (`->`) and then by the **expression** to be used when the value matches the rule.
- The rules are matched in the **order** in which they are defined, and the compiler will issue an error if pattern matching is incomplete.
- The wildcard (`_`) will match **any value** and is a way of telling the compiler that you’re not interested in using this value.
- The `when` guard gives exact control about when a rule executes.


Each pattern is a constructor name followed by the right number of variables (i.e., `C` or `C x` or `C(x,y)` or ...)

- Syntactically most patterns (all today) look like expressions
- But patterns are not expressions
    - We do not evaluate them
    - We see if the result of `e_0` matches them

In [8]:
let booleanToString x =
    match x with
    | false -> "FALSE" 
    | _ -> "TRUE"

sgn booleanToString |> printfn "%s"
    
booleanToString true, booleanToString false

Function: x:bool -> string


Item1,Item2
TRUE,FALSE


In [9]:
let stringToBoolean x = 
    match x with
    | "True" | "true" | "T" | "t" | "1" -> true
    | "False" | "false" | "f" | "0" -> false
    | _ -> failwith "unexpected input string"
    
sgn stringToBoolean |> printfn "%s"    
    
stringToBoolean "True", stringToBoolean "1", stringToBoolean "t"

Function: x:string -> bool


Item1,Item2,Item3
True,True,True


In [10]:
stringToBoolean "Hello!"

Unhandled exception: System.Exception: unexpected input string
   at FSI_0013.stringToBoolean(String x)
   at <StartupCode$FSI_0014>.$FSI_0014.main@()

It is also possible to pattern match over **most of the types** defined by F#.

Pattern matching over tuples:

In [11]:
// let myOr b1 b2 =
//     if      b1 = true  && b2 = true  then true
//     else if b1 = true  && b2 = false then true
//     else if b1 = false && b2 = true  then true
//     else if b1 = false && b2 = false then false
let myOr b1 b2 =
    match (b1, b2) with
    | true, _ -> true
    | _, true -> true
    | _ -> false

In [12]:
(myOr true false), (myOr false false)
// myOr (true, false) // error

Item1,Item2
True,False


In [13]:
// let myAnd b1 b2 =
//     if      b1 = true  && b2 = true  then true
//     else if b1 = false && b2 = true  then false
//     else if b1 = true  && b2 = false then false
//     else if b1 = false && b2 = false then false
// let myAnd b1 b2 =
//     if b1 && b2
//         then true
//         else false
let myAnd p =
    match p with
    | true, true -> true
    | _ -> false
    
sgn myAnd

Function: tupledArg:bool * bool -> bool

In [14]:
myAnd(true, false), myAnd (true, true)
// myAnd true false // problem not tuple

Item1,Item2
False,True


### Why this way is better

0. You can use pattern-matching to write your own testing and data-extractions functions if you must
1. You cannot forget a case (inexhaustive pattern-match warning)
2. You cannot duplicate a case (a type-checking error)
3. You will not forget to test the variant correctly and get an exception (like List.head [])
4. Pattern-matching can be generalized and made more powerful, leading to elegant and concise code
5. Pattern-matching is better for options and lists for the same reasons as for all datatypes
    - No missing cases, no exceptions for wrong variant, etc.


## Pattern matching with expressions

Let's look at the way pattern matching is used for binding values to expressions (the functional equivalent of assigning to variables). See also [conciseness pattern matching](https://fsharpforfunandprofit.com/posts/conciseness-pattern-matching/).

Perform matching directly on tuples:

In [15]:
let firstPart, secondPart, _ =  (1,2,3)  // underscore means ignore
printfn "%i, %i" firstPart secondPart

1, 2


### Lists are datatypes

Do not use `head`, `tail`, or `isEmpty` either
- `[]` and `::` are constructors too
- (strange syntax, particularly infix)

Perform matching directly on lists:

In [16]:
let elem1::elem2::tail = [1..10]       // ignore the warning for now
elem1, elem2, tail


input.fsx (1,5)-(1,23) typecheck warning Incomplete pattern matches on this expression. For example, the value '[_]' may indicate a case not covered by the pattern(s).



Item1,Item2,Item3
1,2,"[ 3, 4, 5, 6, 7, 8, 9, 10 ]"


and with a help of `match..with` construct

In [17]:
let listMatcher aList = 
    match aList with
    | [] -> printfn "the list is empty" 
    | [firstElement] -> printfn "the list has one element %A " firstElement 
    | [first; second] -> printfn "list is %A and %A" first second 
    | head::[b;c] -> printfn "list is three elems: %A, %A and %A" head b c
    | _ -> printfn "the list has more than tree elements"

listMatcher [1;2;3;4]
listMatcher [1;2;3]
listMatcher [1;2]
listMatcher [1]
listMatcher []

the list has more than tree elements
list is three elems: 1, 2 and 3
list is 1 and 2
the list has one element 1 
the list is empty


In [18]:
// list summation using recursion and pattern matching
let rec sum_list_pm (xs : int list) =
    match xs with
    | [] -> 0
    | x::xs -> x + sum_list_pm xs
    
sgn sum_list_pm 

Function: xs:list<int> -> int

In [19]:
sum_list_pm [1; 2; 3; 4; 5; 10]

25

In [20]:
let rec append xs ys =
    match xs with
    | [] -> ys
    | x::xs -> x :: append xs ys

sgn append

Function: xs:list<obj> -> ys:list<obj> -> list<obj>

In [21]:
append [] [1 .. 5],
append [5] [1 .. 5],
append [5 .. 10] [1 .. 5]

Item1,Item2,Item3
"[ 1, 2, 3, 4, 5 ]","[ 5, 1, 2, 3, 4, 5 ]","[ 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5 ]"


Compare to the `sum_list` and `append` without using patter matching

```fsharp
let rec sum_list (xs : int list) = 
    if List.isEmpty xs
    then 0
    else List.head xs + sum_list (List.tail xs)
    
let rec append (xs : int list) (ys : int list) =
    if xs.IsEmpty 
    then ys
    else xs.Head :: append xs.Tail ys    
```

## Exhaustive pattern matching

Here’s some C# code that uses a **switch** statement to handle different types of state.

```c#
enum State { New, Draft, Published, Inactive, Discontinued }
void HandleState(State state)
{
    switch (state)
    {
    case State.Inactive: break; 
    case State.Draft: break;
    case State.New: break;
    case State.Discontinued: break;
    } 
}
```
This code will compile, but there is an obvious **bug**! The compiler couldn’t see it – can you?
If you can, and you fixed it, would it stay fixed if I added another `State` to the list?

Here’s the F# equivalent:

In [22]:
type State = New | Draft | Published | Inactive | Discontinued
let handleState state = 
   match state with
   | Inactive -> () 
   | New -> ()
   | Draft -> ()
   | New -> ()
//    | Published -> ()
   | Discontinued -> ()


input.fsx (3,10)-(3,15) typecheck warning Incomplete pattern matches on this expression. For example, the value 'Published' may indicate a case not covered by the pattern(s).

input.fsx (7,6)-(7,9) typecheck warning This rule will never be matched



The fact that exhaustive matching is always done means that certain common errors will be detected by the compiler immediately:

- A missing case (often caused when a new choice has been added due to changed requirements or refactoring).
- An impossible case (when an existing choice has been removed).
- A redundant case that could never be reached (the case has been subsumed in a previous case – this can sometimes be non-obvious).


### Avoiding nulls with the `Option` type

A typical C# program is littered with code like this:

```c#
if (myObject != null)
{
  // do something
}
```
Unfortunately, this test is not required by the compiler. All it takes is for one piece of code to forget to do this, and the program can crash.
- the invention of `nulls` has even been called a [billion dollar mistake](http://www.infoq.com/presentations/Null-References-The-Billion-Dollar-Mistake-Tony-Hoare)!

In pure F#, nulls cannot exist **accidentally**. A string or object must always be assigned to something at creation, and is immutable thereafter.

### `Options` Type

- the generic wrapper type called `Option`, with two choices: `Some` or `None`.
- `Some` choice wraps a valid value
- `None` represents a missing value

Building:
- `None` has type `'a option` (much like [] has type `'a list`)
- `Some(e)` has type `t option` if `e` has type `t` (much like e::[])

Accessing:
- `.IsSome` has type `'a option -> bool`
- `.Value` has type `'a option -> 'a` (exception if given `None`)
- `.IsNone` has type `'a option -> bool`

In [23]:
let x = Some(10)
x
sgn x, x

Item1,Item2
Value: option<int>,Some(10)


In [24]:
x.IsSome, x.IsNone, x.Value, sgn x.Value

Item1,Item2,Item3,Item4
True,False,10,Value: int


In [25]:
let y = None

y.IsSome, y.IsNone

Item1,Item2
False,True


In [26]:
y.Value

Stopped due to error

input.fsx (1,1)-(1,8) typecheck error Value restriction. The value 'it' has been inferred to have generic type
    val it : obj    
Either define 'it' as a simple data term, make it a function with explicit arguments or, if you do not intend for it to be generic, add a type annotation.



Cell not executed: compilation error

Here's an example where `Some` is returned if a file exists, but a missing file returns `None`.

In [27]:
let getFileInfo filePath =
   let fi = new System.IO.FileInfo(filePath)
   if fi.Exists then Some(fi) else None

In [28]:
let goodFileInfo = getFileInfo "rise.css"
goodFileInfo

Value
rise.css


In [29]:
let badFileInfo = getFileInfo "no-such-file.txt"
badFileInfo, badFileInfo.IsNone

Item1,Item2
<null>,True


If we want to do anything with these values, we must always handle both possible cases.

In [30]:
match goodFileInfo with
| Some fileInfo -> printfn "the file was created at %A" fileInfo.CreationTime
| None -> printfn "the file doesn't exist" 

match badFileInfo with
| Some fileInfo -> printfn "the file was created at %A" fileInfo.CreationTime
| None -> printfn "the file doesn't exist" 

the file was created at 3/25/2020 4:01:24 AM
the file doesn't exist


### Exhaustive pattern matching for edge cases

In [4]:
let rec movingAverages list = 
    match list with
    // if input is empty, return an empty list
    | [] -> []
    // otherwise process pairs of items from the input
    | x::y::rest -> 
        let avg = (x+y)/2.0 
        //build the result by recursing the rest of the list
        avg :: movingAverages (y::rest)
            
movingAverages [1.0]            


input.fsx (2,11)-(2,15) typecheck warning Incomplete pattern matches on this expression. For example, the value '[_]' may indicate a case not covered by the pattern(s).



Unhandled exception: Microsoft.FSharp.Core.MatchFailureException: The match cases were incomplete
   at FSI_0007.movingAverages(FSharpList`1 list)
   at <StartupCode$FSI_0007>.$FSI_0007.main@()

In [5]:
let rec movingAverages list = 
    match list with
    // if input is empty, return an empty list
    | [] -> []
    // otherwise process pairs of items from the input
    | x::y::rest -> 
        let avg = (x+y)/2.0 
        //build the result by recursing the rest of the list
        avg :: movingAverages (y::rest)
    // for one item, return an empty list !!!!
    | [_] -> []

In [6]:
movingAverages [],
movingAverages [1.0],
movingAverages [1.0; 2.0],
movingAverages [1.0 .. 10.0]

Item1,Item2,Item3,Item4
[ ],[ ],[ 1.5 ],"[ 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5 ]"


## Type Pattern Matching

Exhaustive pattern matching is is particularly usefull if you are matching on the cases of a union type:

- Do not to use wildcards, and try to match all the cases explicitly if you can.
- By being always explicit in this way, you can trap any error caused by adding a new case to the union.

In [13]:
type Choices = A | B | C | D

sgn A |> printfn "%A"

match A with 
| A -> "a"
| B -> "b"
| C -> "c"
//NO default match

"Value: Choices"



input.fsx (5,7)-(5,8) typecheck warning Incomplete pattern matches on this expression. For example, the value 'D' may indicate a case not covered by the pattern(s).



a

## Pattern Matching Function

In the examples so far, we’ve seen a lot of this:

In [35]:
let f aValue = 
    match aValue with 
    | x::xs -> sprintf "%i" x
    | _ -> "nothing" 
    
f [1;2], f []

Item1,Item2
1,nothing


In the special case of function definitions we can simplify this dramatically by using the function keyword.

In [21]:
let f = 
    function
    | x::xs -> sprintf "%i" x
    | [] -> "empty"
    
f [1;2], f [2], f []

Item1,Item2,Item3
1,2,empty


- the `aValue` parameter has completely disappeared, along with the `match..with`.
- `function` keyword is not the same as the `fun` keyword for standard lambdas
    - rather it combines `fun` and `match..with` in a single step.

Sometimes pattern matching is just not enough, as we saw in this example:

In [17]:
let elementsAreEqual aTuple = 
//     let x, y = aTuple
    match aTuple with 
    | (x,y) -> 
        if (x=y) then printfn "both parts are the same" 
        else printfn "both parts are different"
        
elementsAreEqual (1, 1)
elementsAreEqual (1, 2) 

both parts are the same
both parts are different


Pattern matching is based on **patterns** only
- it can't use functions or other kinds of conditional tests

But there is a way to do the equality test as part of the pattern match
 - using an additional `when` clause to the left of the function arrow.

In [18]:
let elementsAreEqual aTuple = 
    match aTuple with 
    | (x,y) when x=y && x=1 -> 
        printfn "both parts are the same and 1" 
    | (x,y) when x=y -> 
        printfn "both parts are the same" 
    | _ ->
        printfn "both parts are different" 

elementsAreEqual (1, 1)
elementsAreEqual (2, 2)
elementsAreEqual (1, 2) 

both parts are the same and 1
both parts are the same
both parts are different


## Guards

`when` guard provides control on rule execution

- Composed of the keyword `when` followed by a Boolean expression
- Once the rule is matched, the `when` clause is evaluated, and the rule will fire only if the expression evaluates to `true`.
- If the expression evaluates to `false`, the remaining rules will be searched for another match.

Guards can be used for all sorts of things that pure patterns can’t be used for, such as:

- comparing the bound values
- testing object properties
- doing other kinds of matching, such as regular expressions
- conditionals derived from functions


In [2]:
// The first rule is designed to be the function’s error handler.
// The first part of the rule is an identifier that will match any integer,
// but the `when `guard means the rule will match only those integers that
// are less than or equal to zero

let rec fib x =
    match x with
    | x when x <= 0 -> failwith "value must be greater than 0"
    | 1 -> 1
    | 2 -> 1
    | x -> fib (x - 1) + fib (x - 2)

In [3]:
fib -12

Unhandled exception: System.Exception: value must be greater than 0
   at FSI_0006.fib(Int32 x)
   at <StartupCode$FSI_0007>.$FSI_0007.main@()

## Active patterns

F# has a special type of pattern matching called **"active patterns"** where the pattern can be parsed or detected dynamically.
As with normal patterns, the matching and output are combined into a single step from the caller's point of view, see https://fsharpforfunandprofit.com/posts/convenience-active-patterns/.

- Guards are great for one-off matches
- But if there are certain guards that you use over and over, consider using **active patterns** instead

Here is an example of using active patterns to parse a `string` into an `int` or `bool`.

In [14]:
// create an active pattern
let (|IntAP|_|) (str:string) =
    match System.Int32.TryParse(str) with
    | (true,i) -> Some(i)
    | _ -> None

// create an active pattern
let (|BoolAP|_|) str =
    if str = "true"
    then Some(true)
    else if str = "false" 
    then Some(false)
    else None    

Once these patterns have been set up, they can be used as part of a normal `match..with` expression.

In [16]:
match "127" with
// | s -> printfn "The value is a string '%s'" s
| IntAP i -> printfn "The value is a number '%i'" i
| _ -> printfn "Uknown value"    

The value is a number '127'


In [17]:
match "AAA" with
| IntAP i -> printfn "The value is a number '%i'" i
| _ -> printfn "Uknown value"

Uknown value


In [18]:
match "false" with
| BoolAP b -> printfn "The value is a bool '%b'" b
| _ -> printfn "Uknown value"

The value is a bool 'false'


In [19]:
// create a function to call the patterns
let testParse str = 
    match str with
    | IntAP i -> printfn "The value is an int '%i'" i
    | BoolAP b -> printfn "The value is a bool '%b'" b
    | _ -> printfn "The value '%s' is something else" str

In [22]:
testParse "42"
testParse "false"
testParse "abc"

The value is an int '42'
The value is a bool 'false'
The value 'abc' is something else


### FizzBuzz

Write a program that prints the numbers from 1 to 100. But for multiples of three print "Fizz" instead of the number and for the multiples of five print "Buzz". For numbers which are multiples of both three and five print "FizzBuzz", see https://blog.codinghorror.com/why-cant-programmers-program/.

Most good programmers should be able to write out on paper a program which does this in a under a couple of minutes, right?
- The majority of comp sci graduates can't, even some self-proclaimed senior programmers take more than 10-15 minutes to write a solution.

In [23]:
let fizzBuzz i = 
    let rem3, rem5 = i % 3, i % 5
    if rem3 = 0 then printf "Fizz"
    if rem5 = 0 then printf "Buzz"
    if rem3 <> 0 && rem5 <> 0 then printf "%i" i
    printf ", "
    
[1..100] |> List.iter fizzBuzz

1, 2, Fizz, 4, Buzz, Fizz, 7, 8, Fizz, Buzz, 11, Fizz, 13, 14, FizzBuzz, 16, 17, Fizz, 19, Buzz, Fizz, 22, 23, Fizz, Buzz, 26, Fizz, 28, 29, FizzBuzz, 31, 32, Fizz, 34, Buzz, Fizz, 37, 38, Fizz, Buzz, 41, Fizz, 43, 44, FizzBuzz, 46, 47, Fizz, 49, Buzz, Fizz, 52, 53, Fizz, Buzz, 56, Fizz, 58, 59, FizzBuzz, 61, 62, Fizz, 64, Buzz, Fizz, 67, 68, Fizz, Buzz, 71, Fizz, 73, 74, FizzBuzz, 76, 77, Fizz, 79, Buzz, Fizz, 82, 83, Fizz, Buzz, 86, Fizz, 88, 89, FizzBuzz, 91, 92, Fizz, 94, Buzz, Fizz, 97, 98, Fizz, Buzz, 

In [24]:
// setup the active patterns
let (|MultOf3|_|) i = if i % 3 = 0 then Some MultOf3 else None
let (|MultOf5|_|) i = if i % 5 = 0 then Some MultOf5 else None

// the main function
let fizzBuzzMatch i = 
  match i with
  | MultOf3 & MultOf5 -> printf "FizzBuzz, " 
  | MultOf3 -> printf "Fizz, " 
  | MultOf5 -> printf "Buzz, " 
  | _ -> printf "%i, " i
  
// test
[1..100] |> List.iter fizzBuzzMatch

1, 2, Fizz, 4, Buzz, Fizz, 7, 8, Fizz, Buzz, 11, Fizz, 13, 14, FizzBuzz, 16, 17, Fizz, 19, Buzz, Fizz, 22, 23, Fizz, Buzz, 26, Fizz, 28, 29, FizzBuzz, 31, 32, Fizz, 34, Buzz, Fizz, 37, 38, Fizz, Buzz, 41, Fizz, 43, 44, FizzBuzz, 46, 47, Fizz, 49, Buzz, Fizz, 52, 53, Fizz, Buzz, 56, Fizz, 58, 59, FizzBuzz, 61, 62, Fizz, 64, Buzz, Fizz, 67, 68, Fizz, Buzz, 71, Fizz, 73, 74, FizzBuzz, 76, 77, Fizz, 79, Buzz, Fizz, 82, 83, Fizz, Buzz, 86, Fizz, 88, 89, FizzBuzz, 91, 92, Fizz, 94, Buzz, Fizz, 97, 98, Fizz, Buzz, 